# Robustness of [Newton's Method](https://en.wikipedia.org/wiki/Newton's_method)

### _[Pile item](https://github.com/ArdanaLabs/audit/issues/4)_

_Note_: These are currently preliminary notes ahead of simulation experiments determining upper and lower bounds, values for `A`, etc. 

## Executive summary

Leaving $\chi$ in a black box sabotages any chance at characterizing the polynomial a priori. It is my opinion that we we can best reason about the behavior of the solver if the invariant function is given in polynomials of $D$ and of $x_k$ at a fixed, known value of $\chi$. 

We would like to leverage simulations (and perhaps property tests) to confirm

1. $A > n^{-n}$ (equivalently $An^n > 1$) how often? 
2. Upper bound for $D$.
3. is $D \geq \Sigma x_i$? 

After simulations, we can learn more about 
1. The characterization of some relation between $-A n^{2n} (\Pi x_i) \Sigma x_i$ and $(A - n^{-n})n^{2n} (\Pi x_i)$ (the constant and linear coefficients of $f_D$, respectively) i.e. "$<$ by how much? What are the pathologies where $\geq$ if any?"
2. The characterization of some relation between $\frac{-D^{n+1}}{A n^{2n}\Pi_{i \neq k} x_i}$ and $\Sigma_{i \neq k} x_i + (\frac{1}{A n^n} - 1) D$ (the constant and linear coefficients of $f_k$, respectively) i.e. "$<$ by how much? What are the pathologies where $\geq$ if any?"

This characterization will payout in analyzing the behavior of the solver. Waiting until we learn more about $A$, about the distribution of balances ($x_i$), and about the boundaries around $D$ to provide this characterization could easily multiply the effort. 

# Derivative failures

Rather than divide by zero, [our implementation _guards_ against `f'x` being zero](https://github.com/ArdanaLabs/DanaSwap/blob/b571cbc9eec17f144626f49646e0bf1e14351e3a/src/DanaSwap/Newton.hs#L42). If `f'x` is zero, we return the current guess and end the solver. This introduces a hazard: the derivative evaluated at a guess could be zero well before the guess is adequate. In this section, we discuss avoiding this hazard.

## An ad-hoc strategy

We can prefix the solver with conditional logic that evaluates the zeros of the first derivative. If the zeros satisfy certain properties, we run a more complicated version of the solver that perturbs the value some amount in some direction outside of the usual $x_n - \frac{f x_n}{f' x_n}$. 

**A bonus** of figuring out what the conditional prefix looks like is I found forms of the functions that I think are best for reasoning about 
1. Behavior of the invariant function _as polynomials_ with respect to roots
2. Partial derivatives
3. Plug-and-play different values of $\chi$ in the future. 

## At $\chi = \frac{A (\Pi x_i) n^n}{D^n}$

### The unknown $D$ at $\chi = \frac{A (\Pi x_i) n^n}{D^n}$

**I have run this derivation twice and gotten agreeing results**.

The invariant equation from pg5 of the stableswap paper can be interpreted as a _polynomial function_ $f$ which is set to zero. 

At runtime of the solver, $n$ and all $x_i$ are known. The optimal value for parameter $A$ is discovered by simulation and known by the programmer. 

$$\begin{align}
f &= D \mapsto D^{n+1} + (A - n^{-n})n^{2n} (\Pi x_i) D + -A n^{2n} (\Pi x_i) \Sigma x_i \\
\frac{\partial f}{\partial D} = f' &= D \mapsto (n + 1)D^n + (A - n^{-n})n^{2n} (\Pi x_i) \\
\frac{\partial}{\partial D} \frac{\partial f}{\partial D} = f'' &= D \mapsto (n + 1) n D^{n-1}
\end{align}$$

Where almost surely $A > n^{-n}$

Conveniently for us, $f' D = 0$ can be solved directly (as can $f'' D = 0$)
$$\frac{\partial f}{\partial D} D = 0 \implies D = \left(\frac{(n^{-n} - A) n^{2n} \Pi x_i}{n + 1} \right)^{\frac{1}{n}}$$
where almost surely $A > n^{-n}$, meaning we are taking the `nth` root of a negative number. If $\text{even}\ n$, we can assert that every root of the derivative is complex. When $\text{odd}\ n$, there is one real root, which is negative. Notice we are confident that this root does not equal zero, because the prehoc probability of $A = n^{-n}$ is slim, and in practice value of $A$ is known well before the caller invokes the solver. Still, I'm recommending that $A = n^{-n}$ is a pathology to pay attention to in the simulation stage. 

Pay attention to base function $f$. When $\text{even}\ (n + 1)$ and $-A n^{2n} (\Pi x_i) \Sigma x_i$ "sufficiently low relative to" $(A - n^{-n})n^{2n} (\Pi x_i)$, there is one negative real root and one positive real root. When $\text{odd}\ (n + 1)$ and the same "sufficiently low" condition, there is one positive real root. I believe we could make a quantified proof over most or all of the relevant choices of $A, n, x_i$ that $-A n^{2n} (\Pi x_i) \Sigma x_i < (A - n^{-n})n^{2n} (\Pi x_i)$, but a cursory look at desmos shows that simple $<$ is not always enough. It is, however, very usually enough. Moreover, I'll sign off on a guarantee that $-A n^{2n} (\Pi x_i) \Sigma x_i < 0$ which is enough to guarantee real roots in both even and odd $n$ cases.

### The unknown $x_k$ at $\chi = \frac{A (\Pi x_i) n^n}{D^n}$

**I have run this derivation twice and gotten agreeing results**.

The equation from pg5 of stableswap whitepaper again, interpreted again as a polynomial, but this time with unknown $x_k$

$$\begin{align}
\forall k \in 1..n, f_k &= x_k \mapsto x_k^2 + \left(\Sigma_{i \neq k} x_i + (\frac{1}{A n^n} - 1) D \right) x_k + \frac{-D^{n+1}}{A n^{2n}\Pi_{i \neq k} x_i} \\
\forall k \in 1..n, \frac{\partial f_k}{\partial x_k} = f_k' &= x_k \mapsto 2 x_k + \left(\Sigma_{i \neq k} x_i + (\frac{1}{A n^n} - 1) D \right) \\
\forall k \in 1..n, \frac{\partial}{\partial x_k} \frac{\partial f_k}{\partial x_k} = f_k'' &= x_k \mapsto 2
\end{align}$$

If $\mathbb{R}^+$ is an approximation of the domain of balances, we should have $\forall (i,j) \in 1..n \times 1..n, \forall x \in \mathbb{R}^+, f_i x = f_j x$, where the difference between $f_k$s is merely symbolic so the partials become easier to write. 

Similarly, we shall want to solve for $\frac{\partial f_k}{\partial x_k} x_k = 0$

$$f_k' x_k = 0 \implies x_k = \frac{1}{2} \left((1 - \frac{1}{A n^n}) D - \Sigma_{i \neq k} \right)$$

Where almost surely $A n^n > 1$, so we can think of $D$ as being weighted down, in other words $(1 - \frac{1}{A n^n}) D \in ]0, D[$. Thus, the sign of the unique root is determined entirely by comparator judgment $(1 - \frac{1}{A n^n}) D$ vs. $\Sigma_{i \neq k} x_i$. In any case, $A n^n \leq 1$ is a pathology that we should learn about in the simulation stage.

If $\Sigma_{i \neq k} x_i = (1 - \frac{1}{A n^n}) D$, then the solver will dubiously converge via the logic that prevents division by zero. We want to avoid this outcome. 

If $\Sigma_{i \neq k} x_i < (1 - \frac{1}{A n^n}) D$, then the root of the derivative is positive, 

If $\Sigma_{i \neq k} x_i > (1 - \frac{1}{A n^n}) D$, then the root of the derivative is negative, and we don't have to worry. 

Regarding the base function, once again our constant is guaranteed negative and our linear term is guaranteed positive, so we should strongly conjecture that there will be one positive real root. 

### The recommendation if hardcoding a known value of $\chi$ is allowable

implement **both** `invariantInD` and `invariantInXi`, along with their respective partials, as written above. The equivalence relation joining them is $\sim\ = \{(f, g) | \text{f and g have the same zeros} \}$, which could be unit tested for but I'm not sure how to property test it. 

## At unknown, arbitrary $\chi$

The unknown $\chi$ is dependent on both $D$ and $x_k$ for each $k \in 1..n$. Here, we get zero payout in understanding the polynomial's zero behavior, since relevant terms are hidden in a black box. 

### The unknown $D$ 

**Not double checked as of this writing but I'm not worried about this one as it's super easy**. 

Here is the invariant function grouped in the $D$ terms _that we can see_. Its partial is calculated with the product rule then grouped in the $D$ terms _that we can see_. 

The payout to grouping even though we can't see all the relevant terms is the form of the partial. 

$$\begin{align}
f &= D \mapsto (\chi + n^{-n}) D^n - (\Sigma x_i) \chi D^{n - 1} - \Pi x_i \\
\frac{\partial f}{\partial D} = f' &= D \mapsto \frac{\partial \chi}{\partial D} D^n + \left( (\chi + n^{-n}) n - (\Sigma x_i) \frac{\partial \chi}{\partial D} \right) D^{n-1} + (1 - n)(\Sigma x_i) D^{n-2}
\end{align}$$

I can't say anything about the roots of the derivative for arbitrary $n$. If set to zero, we can make the derivative appear quadratic by dividing out $D^{n-2}$, but this is false as there are more $D$ terms in $\chi$ and $\frac{\partial \chi}{\partial D}$.

### The unknown $x_k$ 

**Not double checked as of this writing**.

Here is the invariant function grouped in the $x_k$ terms _that we can see_. The partial is calculated with the product rule then grouped in the $x_k$ terms _that we can see_. 

The payout to grouping even though we can't see all the relevant terms is the form of the partial. 

$$\begin{align}
\forall k \in 1..n, f_k &= x_k \mapsto (D^{n-1} \chi + \Pi_{i \neq k} x_i) x_k + \left((D^{n-1} (\Sigma_{i \neq k} x_i) - D^n) \chi - \frac{D^n}{n^n} \right)\\
\forall k \in 1..n, \frac{\partial f_k}{x_k} = f_k' &= x_k \mapsto D^{n-1} \frac{\partial \chi}{\partial x_k} x_k + D^{n-1} \chi + (\Sigma_{i \neq k} x_i - D) D^{n-1} \frac{\partial \chi}{\partial x_k} + \Pi_{i \neq k} x_i
\end{align}$$

Conveniently, we can compute the zero of the first derivative a priori. 

$$ f_k' x_k = 0 \implies x_k = \frac{-(\chi + \Pi_{i \neq k} x_i)}{\frac{\partial \chi}{\partial x_k}} + D - \Sigma_{i \neq k} x_i$$

However unlikely it is that $\Pi_{i \neq k} x_i = - \chi$ and $D = \Sigma_{i \neq k} x_i$ at the same time, or otherwise something that would zero out the derivative, we know that the solver will halt dubiously at $\frac{\partial \chi}{\partial x_k} = 0$, which is also unlikely based on everything I know about the principles of selecting a leverage ($\chi$) function. 

# Further analysis of solver behavior

Should review [convergence proof techniques from analysis](https://math.stackexchange.com/questions/2501882/how-to-prove-a-recursive-sequence-converges), perhaps a direct application quantified in terms of balances. 

Consider the invariant in $D$, polynomial function $f$. We want to scrutinize properties that the coefficients have to satisfy in order for newton's method to converge.

Rename $D$ to $x$. 

$$\forall n \in \mathbb{N} \setminus \{0, 1\}, f := x \mapsto x^{n+1} + ax + b : [l,r] \rightarrow \mathbb{R}; f' := x \mapsto (n + 1)x^n + a$$

We define the sequence $x_k$. 

$$
x_k :=
    \begin{cases}
    BC & x_0 = x_0 \\
    RS & \forall k \in \mathbb{N}, x_{k+1} = x_k - \frac{x_k^{n+1} + ax + b}{(n + 1) x_k^n + a} = \frac{n x_k^{n+1} - b}{(n + 1) x_k^n + a}
    \end{cases}
$$



$$
\text{beliefs} := \begin{cases}
    b < 0 < a \\
    0 < l < r \\
    \forall m \in \mathbb{N}, x_m \in [l,r]\ (\text{preservation of domain on sequence path})
\end{cases}
\\
\text{desire} := \begin{cases}
    \exists c \in [l,r], \forall \epsilon > 0, \exists j \in \mathbb{N}, \forall i > j-1, |c - x_{i+1}| < \epsilon\ (\text{convergence})
\end{cases}
$$

Let's start with what we desire, figure out what our desire entails, and work backwards to figure out what assumptions we must impose to accomplish our desire. 

$$
|c - x_{i+1}| < \epsilon \implies |c| - |x_{i+1}| \leq |c - x_{i+1}| < \epsilon \implies c - x_{i + 1} < \epsilon
$$

### ... analysis techniques for recursive sequences have been a wild goose chase after a several-hours pass. Probably not worth getting a more talented analyst in here (but I do know a guy) 

Though I did get a sense that _preservation of domain on sequence path_ is unlikely to be true, which as far as I can tell would have been our only hope at proving boundedness of the sequence, a really nice property. 

# Idea

We should think about this from the perspective of **economic impact of precision choices**. 

There are two precision choices to make: $\delta$ such that $|x_k - x_{k+1}| < \delta$ is when we stop, and $\epsilon$ such that $f(x_k) < \epsilon$ is considered sufficient. 

# Totally new idea: screw newton's method

Use a matrix solver to find eigenvalues of the invariant polynomial's [companion matrix](https://en.wikipedia.org/wiki/Companion_matrix), because by definition the characteristic polynomial of the companion matrix is the polynomial. 

That way we could leverage [`eigen`](https://hackage.haskell.org/package/eigen) or some other matrix library, though we should probably audit the library. 

Notably, the sparseness of companion matrices grows quickly with $n$. 

## Morgan recommended against this at this time. See [here](https://github.com/ArdanaLabs/audit/issues/4#issuecomment-926091254) for details

In [3]:
import sympy

